In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls ..

In [ ]:
!pwd

In [ ]:
import pandas as pd
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch

In [ ]:
import zipfile

path_to_zip_file = "../input/train.zip"
directory_to_extract_to = "./"


#simple function to extract the train data
def extract(zip_file, path):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(path)


In [ ]:
#extract('/kaggle/input/text-classification/github.tar', '/kaggle/working/datasets')

# Data tokenization

In [ ]:
train = pd.read_json('../input/text-classification/github/embold_train.json')
train_extra = pd.read_json('../input/text-classification/github/embold_train_extra.json')
train = pd.concat([train, train_extra], ignore_index=True)
test = pd.read_json('../input/text-classification/github/embold_test.json')
# 0: bug, 1: feature, 2: question

In [ ]:
train['text'] = train.title + ' ' + train.body
test['text'] = test.title + ' ' + test.body

train.head()

In [ ]:
print(len(train), len(test))

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=.2)

In [ ]:
train_texts = train[:20_000].text.to_list()#to_numpy()
val_texts = val[:15_000].text.to_list()#to_numpy()
test_texts = val[-15_000:].text.to_list()
#test_texts = test[:2000].text.to_list()#to_numpy()

In [ ]:
train_labels = train[:20_000].label.to_list()#to_numpy()
val_labels = val[:15_000].label.to_list()#to_numpy()
test_labels = val[-15_000:].label.to_list()
#test_labels = test[:2000].label.to_list()#to_numpy()

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# Datasets

In [ ]:
import torch

class GitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GitDataset(train_encodings, train_labels)
val_dataset = GitDataset(val_encodings, val_labels)
test_dataset = GitDataset(test_encodings, test_labels)

# Fine-tuning

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        #'f1': f1,
        #'precision': precision,
        #'recall': recall
    }


In [ ]:
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics = compute_metrics
)

In [ ]:
model.to('cuda')

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('bert_github_kaggle')

In [ ]:
print("End")